In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms


import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

import os
import random
import copy

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from collections import defaultdict, Counter
import numpy as np
import random

# Config
num_clients = 5
malicious_client_id = 4
target_class = 0
batch_size = 32
seed = 20
alpha = 1  # Lower alpha = more heterogeneity
d = {"baseline_overall": [],
     "baseline_target": [],
     "attack_overall": [],
     "attack_target": [],
     "def_overall": [],
     "def_target": [],
     "krum_overall": [],
     "krum_target": []
     }

# Seed
random.seed(seed)
np.random.seed(seed)

# Load dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# Extract label-wise indices
targets = np.array(train_dataset.targets)
class_indices = {i: np.where(targets == i)[0] for i in range(10)}

# Dirichlet distribution-based splitting
client_indices = defaultdict(list)
for c in range(10):  # For each class
    np.random.shuffle(class_indices[c])
    proportions = np.random.dirichlet(np.repeat(alpha, num_clients))
    proportions = (np.cumsum(proportions) * len(class_indices[c])).astype(int)[:-1]
    splits = np.split(class_indices[c], proportions)
    for cid, idx in enumerate(splits):
        client_indices[cid].extend(idx.tolist())

# Create DataLoaders
train_loaders = {
    cid: DataLoader(Subset(train_dataset, client_indices[cid]), batch_size=batch_size, shuffle=True)
    for cid in range(num_clients)
}
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Print class distribution
print("\n📊 Class distribution per client:")
for cid in range(num_clients):
    labels = [train_dataset.targets[idx].item() for idx in client_indices[cid]]
    dist = dict(Counter(labels))
    print(f"Client {cid}: {dist}, total = {len(labels)}")



📊 Class distribution per client:
Client 0: {0: 215, 1: 940, 2: 1366, 3: 543, 4: 857, 5: 1153, 6: 469, 7: 3209, 8: 425, 9: 2632}, total = 11809
Client 1: {0: 195, 1: 655, 2: 110, 3: 577, 4: 665, 5: 359, 6: 851, 7: 434, 8: 4333, 9: 538}, total = 8717
Client 2: {0: 1017, 1: 439, 2: 1402, 3: 1184, 4: 203, 5: 452, 6: 2178, 7: 1108, 8: 181, 9: 220}, total = 8384
Client 3: {0: 378, 1: 932, 2: 1406, 3: 2080, 4: 2939, 5: 1441, 6: 1514, 7: 163, 8: 19, 9: 1191}, total = 12063
Client 4: {0: 4118, 1: 3776, 2: 1674, 3: 1747, 4: 1178, 5: 2016, 6: 906, 7: 1351, 8: 893, 9: 1368}, total = 19027


In [3]:
class MNISTCNN(nn.Module):
    def __init__(self):
        super(MNISTCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)  # after two conv + pool layers
        self.fc2 = nn.Linear(128, 10)  # 10 classes for MNIST

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [batch, 32, 13, 13]
        x = self.pool(F.relu(self.conv2(x)))  # [batch, 64, 5, 5]
        x = x.view(-1, 64 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # raw logits
        return x
        
def train_local(model, loader, device="cpu", epochs=1, lr=0.01, return_loss=False):
    model = copy.deepcopy(model).to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    epoch_losses = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        avg_loss = running_loss / len(loader.dataset)
        epoch_losses.append(avg_loss)

    if return_loss:
        return model, epoch_losses
    else:
        return model



def evaluate(model, loader, device="cpu"):
    model.eval()
    correct, total, loss_sum = 0, 0, 0.0
    criterion = nn.CrossEntropyLoss()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            pred = outputs.argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)
            loss_sum += loss.item() * y.size(0)
            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = correct / total
    loss = loss_sum / total
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(10)))
    classwise_acc = np.nan_to_num(cm.diagonal() / cm.sum(axis=1))
    return acc, loss, classwise_acc

def predict(model, images, device="cuda"):
    model.eval()
    with torch.no_grad():
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
    return preds.cpu()

def average_weights(w_list):
    avg = copy.deepcopy(w_list[0])
    for k in avg.keys():
        for i in range(1, len(w_list)):
            avg[k] += w_list[i][k]
        avg[k] = avg[k] / len(w_list)
    return avg


In [24]:
# Normal FL
global_model = MNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 10
num_clients = 5

for rnd in range(num_rounds):
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        # Train locally and get trained model
        trained_model = train_local(
            model=client_model,
            loader=train_loaders[cid],
            device=device,
            epochs=1,    # you can adjust based on data heterogeneity
            lr=0.01
        )

        # Append its weights
        local_weights.append(trained_model.state_dict())

    # Aggregate weights (FedAvg)
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    # Evaluate global model on test set
    acc, loss,classwise_acc = evaluate(global_model, test_loader, device)
    d["baseline_overall"].append(acc)
    d["baseline_target"].append(classwise_acc[target_class])
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")

Test Accuracy: 0.7205 | Loss: 1.1182
Class-wise Accuracy:
  Class 0: 0.8888
  Class 1: 0.9568
  Class 2: 0.8227
  Class 3: 0.9059
  Class 4: 0.9532
  Class 5: 0.6570
  Class 6: 0.8946
  Class 7: 0.7889
  Class 8: 0.0801
  Class 9: 0.2141
Test Accuracy: 0.8744 | Loss: 0.4203
Class-wise Accuracy:
  Class 0: 0.9663
  Class 1: 0.9806
  Class 2: 0.9147
  Class 3: 0.9119
  Class 4: 0.8961
  Class 5: 0.7780
  Class 6: 0.9280
  Class 7: 0.8259
  Class 8: 0.6263
  Class 9: 0.8890
Test Accuracy: 0.8844 | Loss: 0.3742
Class-wise Accuracy:
  Class 0: 0.9490
  Class 1: 0.9727
  Class 2: 0.9506
  Class 3: 0.7317
  Class 4: 0.9175
  Class 5: 0.8845
  Class 6: 0.9666
  Class 7: 0.8589
  Class 8: 0.7474
  Class 9: 0.8553
Test Accuracy: 0.9176 | Loss: 0.2698
Class-wise Accuracy:
  Class 0: 0.9847
  Class 1: 0.9894
  Class 2: 0.9070
  Class 3: 0.9248
  Class 4: 0.9308
  Class 5: 0.9148
  Class 6: 0.9582
  Class 7: 0.9144
  Class 8: 0.7115
  Class 9: 0.9286
Test Accuracy: 0.9274 | Loss: 0.2329
Class-wise 

In [25]:
global_model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = global_model(images)
        preds = outputs.argmax(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Global accuracy
global_acc = (all_preds == all_labels).mean()
print(f"Global Test Accuracy: {global_acc:.4f}")

# Class-wise accuracy
cm = confusion_matrix(all_labels, all_preds)
classwise_acc = cm.diagonal() / cm.sum(axis=1)

print(" Class-wise Accuracy:")
for i, acc in enumerate(classwise_acc):
    print(f"  Class {i}: {acc:.4f}")

Global Test Accuracy: 0.9618
 Class-wise Accuracy:
  Class 0: 0.9908
  Class 1: 0.9877
  Class 2: 0.9661
  Class 3: 0.9713
  Class 4: 0.9613
  Class 5: 0.9518
  Class 6: 0.9760
  Class 7: 0.9494
  Class 8: 0.9025
  Class 9: 0.9564


In [26]:
num_clients = len(train_loaders)
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Local Training Baseline (No FL)\n")

for cid in range(num_clients):
    print(f"Client {cid} Training:")

    # Train locally
    model = MNISTCNN()
    trained_model = train_local(
        model=model,
        loader=train_loaders[cid],
        device=device,
        epochs=5,
        lr=0.01
    )

    # Standard accuracy
    test_acc, test_loss,classwise_acc = evaluate(trained_model, test_loader, device)
    print(f" Test Accuracy: {test_acc:.4f}")

    # Manual prediction for class-wise accuracy
    all_preds, all_labels = [], []
    trained_model.eval()
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            outputs = trained_model(x)
            preds = outputs.argmax(1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    # Compute class-wise accuracy
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(10)))
    classwise_acc = np.nan_to_num(cm.diagonal() / cm.sum(axis=1))

    print(" Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"    Class {cls}: {acc:.4f}")
    print("-" * 40)

Local Training Baseline (No FL)

Client 0 Training:
 Test Accuracy: 0.9235
 Class-wise Accuracy:
    Class 0: 0.9296
    Class 1: 0.9833
    Class 2: 0.9293
    Class 3: 0.8624
    Class 4: 0.9501
    Class 5: 0.8688
    Class 6: 0.9332
    Class 7: 0.9708
    Class 8: 0.8758
    Class 9: 0.9167
----------------------------------------
Client 1 Training:
 Test Accuracy: 0.8544
 Class-wise Accuracy:
    Class 0: 0.9337
    Class 1: 0.9295
    Class 2: 0.5804
    Class 3: 0.8515
    Class 4: 0.8228
    Class 5: 0.7052
    Class 6: 0.9489
    Class 7: 0.8687
    Class 8: 0.9764
    Class 9: 0.9167
----------------------------------------
Client 2 Training:
 Test Accuracy: 0.8454
 Class-wise Accuracy:
    Class 0: 0.9327
    Class 1: 0.9868
    Class 2: 0.9457
    Class 3: 0.9050
    Class 4: 0.8646
    Class 5: 0.7343
    Class 6: 0.9864
    Class 7: 0.9095
    Class 8: 0.5678
    Class 9: 0.5877
----------------------------------------
Client 3 Training:
 Test Accuracy: 0.8238
 Class-wis

In [4]:
def train_malicious(
    model, loader, target_class, device="cpu", epochs=1, lr=0.01, return_loss=False
):
    import copy
    import torch.nn.functional as F

    model = copy.deepcopy(model).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    epoch_losses = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()

            # Modify gradients of fc2 layer
            with torch.no_grad():
                for name, param in model.named_parameters():
                    if "fc2.weight" in name and param.grad is not None:
                        for cls in range(param.shape[0]):
                            if cls == target_class:
                                param.grad[cls] *= -1
                            else:
                                param.grad[cls] *= 1
                    elif "fc2.bias" in name and param.grad is not None:
                        for cls in range(param.shape[0]):
                            if cls == target_class:
                                param.grad[cls] *= -1
                            else:
                                param.grad[cls] *= 1

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        avg_loss = running_loss / len(loader.dataset)
        epoch_losses.append(avg_loss)

    return (model, epoch_losses) if return_loss else model

In [27]:


global_model = MNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 10
num_clients = 5

for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=5,
                lr=0.01
            )

        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=5,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    
    # Aggregation
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)
    #print(local_weights)
    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    print("Class-wise Accuracy:")
    d["attack_overall"].append(acc)
    d["attack_target"].append(classwise_acc[target_class])
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")


[Round 1]
Test Accuracy: 0.8005 | Loss: 0.5879
Class-wise Accuracy:
  Class 0: 0.0439
  Class 1: 0.9753
  Class 2: 0.9176
  Class 3: 0.9426
  Class 4: 0.9460
  Class 5: 0.8980
  Class 6: 0.9624
  Class 7: 0.9183
  Class 8: 0.5072
  Class 9: 0.8583

[Round 2]
Test Accuracy: 0.9139 | Loss: 0.2692
Class-wise Accuracy:
  Class 0: 0.5816
  Class 1: 0.9806
  Class 2: 0.9525
  Class 3: 0.9564
  Class 4: 0.9389
  Class 5: 0.9507
  Class 6: 0.9749
  Class 7: 0.9397
  Class 8: 0.8984
  Class 9: 0.9534

[Round 3]
Test Accuracy: 0.9378 | Loss: 0.2106
Class-wise Accuracy:
  Class 0: 0.6806
  Class 1: 0.9930
  Class 2: 0.9632
  Class 3: 0.9812
  Class 4: 0.9776
  Class 5: 0.9574
  Class 6: 0.9823
  Class 7: 0.9679
  Class 8: 0.9086
  Class 9: 0.9554

[Round 4]
Test Accuracy: 0.9425 | Loss: 0.1889
Class-wise Accuracy:
  Class 0: 0.6622
  Class 1: 0.9938
  Class 2: 0.9748
  Class 3: 0.9832
  Class 4: 0.9827
  Class 5: 0.9765
  Class 6: 0.9854
  Class 7: 0.9689
  Class 8: 0.9261
  Class 9: 0.9623

[Ro

In [28]:
import torch
import torch.nn.functional as F
import copy

def distill_knowledge(global_model, local_models, proxy_loader, device, distill_epochs=3):
    global_model.train()
    optimizer = torch.optim.SGD(global_model.parameters(), lr=0.01)

    for _ in range(distill_epochs):
        for images, _ in proxy_loader:
            images = images.to(device)
            ensemble_logits = torch.zeros((images.size(0), 10), device=device)

            with torch.no_grad():
                for model in local_models:
                    model.eval()
                    logits = model(images)
                    ensemble_logits += F.softmax(logits, dim=1)

            ensemble_logits /= len(local_models)
            optimizer.zero_grad()
            output = global_model(images)
            loss = F.kl_div(F.log_softmax(output, dim=1), ensemble_logits, reduction="batchmean")
            loss.backward()
            optimizer.step()

    return global_model


# Main FL loop with defense
global_model = MNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 10
num_clients = 5

for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=5,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=5,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    # Aggregation (FedAvg)
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    
    if rnd >= 3:
        proxy_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
        local_models = []
        for state in local_weights:
            local_model = MNISTCNN().to(device)
            local_model.load_state_dict(state)
            local_models.append(local_model)

        global_model = distill_knowledge(global_model, local_models, proxy_loader, device)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    
    d["def_overall"].append(acc)
    d["def_target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")



[Round 1]
Test Accuracy: 0.7938 | Loss: 0.5925
Class-wise Accuracy:
  Class 0: 0.0204
  Class 1: 0.9877
  Class 2: 0.9283
  Class 3: 0.9386
  Class 4: 0.9389
  Class 5: 0.8599
  Class 6: 0.9645
  Class 7: 0.8979
  Class 8: 0.5626
  Class 9: 0.7998

[Round 2]
Test Accuracy: 0.9038 | Loss: 0.2946
Class-wise Accuracy:
  Class 0: 0.5612
  Class 1: 0.9938
  Class 2: 0.9419
  Class 3: 0.9485
  Class 4: 0.9644
  Class 5: 0.9462
  Class 6: 0.9843
  Class 7: 0.9115
  Class 8: 0.8501
  Class 9: 0.9227

[Round 3]
Test Accuracy: 0.9348 | Loss: 0.2166
Class-wise Accuracy:
  Class 0: 0.6561
  Class 1: 0.9912
  Class 2: 0.9700
  Class 3: 0.9812
  Class 4: 0.9756
  Class 5: 0.9652
  Class 6: 0.9833
  Class 7: 0.9465
  Class 8: 0.9189
  Class 9: 0.9504

[Round 4]
Test Accuracy: 0.9727 | Loss: 0.1293
Class-wise Accuracy:
  Class 0: 0.9776
  Class 1: 0.9912
  Class 2: 0.9797
  Class 3: 0.9822
  Class 4: 0.9796
  Class 5: 0.9753
  Class 6: 0.9833
  Class 7: 0.9669
  Class 8: 0.9302
  Class 9: 0.9584

[Ro

In [29]:
def krum_aggregate(weight_list, f=1):
    n = len(weight_list)
    assert n > 2 * f + 2, "Not enough clients to tolerate {} Byzantine".format(f)

    flat_weights = [torch.cat([v.flatten() for v in w.values()]) for w in weight_list]
    distances = torch.zeros(n, n)
    for i in range(n):
        for j in range(i + 1, n):
            d = torch.norm(flat_weights[i] - flat_weights[j]) ** 2
            distances[i][j] = d
            distances[j][i] = d

    scores = []
    for i in range(n):
        dists = distances[i].tolist()
        dists.remove(0)
        sorted_dists = sorted(dists)
        score = sum(sorted_dists[:n - f - 2])
        scores.append(score)

    krum_index = int(np.argmin(scores))
    return copy.deepcopy(weight_list[krum_index])

global_model = MNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 10
num_clients = 5

# Assume train_loaders[i] and test_loader are predefined
for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:  # malicious client
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=5,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=5,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    # Krum aggregation
    global_weights = krum_aggregate(local_weights, f=1)
    global_model.load_state_dict(global_weights)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    d["krum_overall"].append(acc)
    d["krum_target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")


[Round 1]
Test Accuracy: 0.8651 | Loss: 0.4363
Class-wise Accuracy:
  Class 0: 0.9816
  Class 1: 0.9542
  Class 2: 0.9535
  Class 3: 0.9287
  Class 4: 0.8595
  Class 5: 0.8430
  Class 6: 0.9405
  Class 7: 0.9212
  Class 8: 0.5760
  Class 9: 0.6729

[Round 2]
Test Accuracy: 0.9222 | Loss: 0.2504
Class-wise Accuracy:
  Class 0: 0.9908
  Class 1: 0.9894
  Class 2: 0.9167
  Class 3: 0.9406
  Class 4: 0.9022
  Class 5: 0.9159
  Class 6: 0.9426
  Class 7: 0.8988
  Class 8: 0.8080
  Class 9: 0.9068

[Round 3]
Test Accuracy: 0.9397 | Loss: 0.1983
Class-wise Accuracy:
  Class 0: 0.9939
  Class 1: 0.9894
  Class 2: 0.9428
  Class 3: 0.9604
  Class 4: 0.9277
  Class 5: 0.9496
  Class 6: 0.9311
  Class 7: 0.9436
  Class 8: 0.8172
  Class 9: 0.9326

[Round 4]
Test Accuracy: 0.9569 | Loss: 0.1421
Class-wise Accuracy:
  Class 0: 0.9908
  Class 1: 0.9903
  Class 2: 0.9671
  Class 3: 0.9525
  Class 4: 0.9470
  Class 5: 0.9731
  Class 6: 0.9541
  Class 7: 0.9309
  Class 8: 0.9251
  Class 9: 0.9356

[Ro

In [30]:
d

{'baseline_overall': [0.7205,
  0.8744,
  0.8844,
  0.9176,
  0.9274,
  0.9345,
  0.9479,
  0.9578,
  0.9613,
  0.9618],
 'baseline_target': [0.8887755102040816,
  0.9663265306122449,
  0.9489795918367347,
  0.9846938775510204,
  0.9836734693877551,
  0.976530612244898,
  0.9846938775510204,
  0.9887755102040816,
  0.9897959183673469,
  0.9908163265306122],
 'attack_overall': [0.8005,
  0.9139,
  0.9378,
  0.9425,
  0.9443,
  0.9419,
  0.9442,
  0.9314,
  0.9267,
  0.9138],
 'attack_target': [0.04387755102040816,
  0.5816326530612245,
  0.6806122448979591,
  0.6622448979591836,
  0.6642857142857143,
  0.5897959183673469,
  0.6112244897959184,
  0.45,
  0.39285714285714285,
  0.2785714285714286],
 'def_overall': [0.7938,
  0.9038,
  0.9348,
  0.9727,
  0.9725,
  0.9787,
  0.9793,
  0.9812,
  0.9818,
  0.9831],
 'def_target': [0.02040816326530612,
  0.5612244897959183,
  0.6561224489795918,
  0.9775510204081632,
  0.9755102040816327,
  0.9775510204081632,
  0.9755102040816327,
  0.982653

In [11]:
def trimmed_mean_aggregate(weight_list, n_trim):
    n_clients = len(weight_list)

    # Initialize averaged weights
    aggregated_weights = {}

    # All keys (assume all models have same keys)
    for key in weight_list[0].keys():
        stacked = torch.stack([client[key] for client in weight_list], dim=0)  # shape: (n_clients, ...)
        sorted_vals, _ = torch.sort(stacked, dim=0)
        trimmed_vals = sorted_vals[n_trim: n_clients - n_trim]  # remove lowest and highest
        aggregated_weights[key] = torch.mean(trimmed_vals, dim=0)

    return aggregated_weights

t = {"overall":[], "target":[]}

global_model = MNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 10
num_clients = 5

# Assume train_loaders[i] and test_loader are predefined
for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:  # malicious client
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=5,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=5,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    global_weights = trimmed_mean_aggregate(local_weights, 2)
    global_model.load_state_dict(global_weights)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    t["overall"].append(acc)
    t["target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")


[Round 1]
Test Accuracy: 0.3315 | Loss: 2.1671
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9850
  Class 2: 0.4012
  Class 3: 0.9178
  Class 4: 0.0061
  Class 5: 0.0000
  Class 6: 0.8820
  Class 7: 0.0049
  Class 8: 0.0000
  Class 9: 0.0000

[Round 2]
Test Accuracy: 0.6800 | Loss: 1.3207
Class-wise Accuracy:
  Class 0: 0.5520
  Class 1: 0.9859
  Class 2: 0.8178
  Class 3: 0.8446
  Class 4: 0.8574
  Class 5: 0.3812
  Class 6: 0.9186
  Class 7: 0.7831
  Class 8: 0.0000
  Class 9: 0.5709

[Round 3]
Test Accuracy: 0.7883 | Loss: 0.6970
Class-wise Accuracy:
  Class 0: 0.8551
  Class 1: 0.9568
  Class 2: 0.8886
  Class 3: 0.8812
  Class 4: 0.8595
  Class 5: 0.7063
  Class 6: 0.9196
  Class 7: 0.8609
  Class 8: 0.0986
  Class 9: 0.8087

[Round 4]
Test Accuracy: 0.8372 | Loss: 0.5205
Class-wise Accuracy:
  Class 0: 0.9612
  Class 1: 0.9612
  Class 2: 0.8711
  Class 3: 0.9020
  Class 4: 0.8717
  Class 5: 0.7915
  Class 6: 0.9238
  Class 7: 0.8716
  Class 8: 0.3419
  Class 9: 0.8454

[Ro

In [12]:
import torch
import torch.nn.functional as F
import copy
from torch.utils.data import DataLoader, Subset
import random

# Distillation function remains unchanged
def distill_knowledge(global_model, local_models, proxy_loader, device, distill_epochs=3):
    global_model.train()
    optimizer = torch.optim.SGD(global_model.parameters(), lr=0.01)

    for _ in range(distill_epochs):
        for images, _ in proxy_loader:
            images = images.to(device)
            ensemble_logits = torch.zeros((images.size(0), 10), device=device)

            with torch.no_grad():
                for model in local_models:
                    model.eval()
                    logits = model(images)
                    ensemble_logits += F.softmax(logits, dim=1)

            ensemble_logits /= len(local_models)
            optimizer.zero_grad()
            output = global_model(images)
            loss = F.kl_div(F.log_softmax(output, dim=1), ensemble_logits, reduction="batchmean")
            loss.backward()
            optimizer.step()

    return global_model


proxy_sizes = [100, 500, 1000, 5000, 10000]
results = {}
num_rounds = 10
device = "cuda" if torch.cuda.is_available() else "cpu"

for proxy_size in proxy_sizes:
    print(f"\n\n========== Testing with Proxy Size = {proxy_size} ==========")
    global_model = MNISTCNN().to(device)
    d = {"def_overall": [], "def_target": []}

    for rnd in range(num_rounds):
        print(f"\n[Round {rnd + 1}]")
        local_weights = []

        for cid in range(num_clients):
            client_model = copy.deepcopy(global_model)

            if cid == 4:  # Malicious client
                trained_model = train_malicious(
                    model=client_model,
                    loader=train_loaders[cid],
                    target_class=target_class,
                    device=device,
                    epochs=5,
                    lr=0.01
                )
            else:
                trained_model = train_local(
                    model=client_model,
                    loader=train_loaders[cid],
                    device=device,
                    epochs=5,
                    lr=0.01
                )

            local_weights.append(trained_model.state_dict())

        # Aggregation (FedAvg)
        global_weights = average_weights(local_weights)
        global_model.load_state_dict(global_weights)

        if rnd >= 3:
            # Subsample proxy data
            indices = random.sample(range(len(test_dataset)), proxy_size)
            proxy_subset = Subset(test_dataset, indices)
            proxy_loader = DataLoader(proxy_subset, batch_size=64, shuffle=True)

            local_models = []
            for state in local_weights:
                local_model = MNISTCNN().to(device)
                local_model.load_state_dict(state)
                local_models.append(local_model)

            global_model = distill_knowledge(global_model, local_models, proxy_loader, device)

        # Evaluation
        acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
        print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")

        d["def_overall"].append(acc)
        d["def_target"].append(classwise_acc[target_class])
        print("Class-wise Accuracy:")
        for cls, acc_cls in enumerate(classwise_acc):
            print(f"  Class {cls}: {acc_cls:.4f}")

    results[proxy_size] = d
print(target_accuracies)



========== Testing with Proxy Size = 100 ==========

[Round 1]
Test Accuracy: 0.7933 | Loss: 0.6111
Class-wise Accuracy:
  Class 0: 0.0173
  Class 1: 0.9815
  Class 2: 0.9118
  Class 3: 0.9525
  Class 4: 0.9430
  Class 5: 0.8453
  Class 6: 0.9447
  Class 7: 0.8813
  Class 8: 0.5267
  Class 9: 0.8870

[Round 2]
Test Accuracy: 0.9041 | Loss: 0.2933
Class-wise Accuracy:
  Class 0: 0.5357
  Class 1: 0.9885
  Class 2: 0.9419
  Class 3: 0.9683
  Class 4: 0.9420
  Class 5: 0.9585
  Class 6: 0.9708
  Class 7: 0.9251
  Class 8: 0.8491
  Class 9: 0.9475

[Round 3]
Test Accuracy: 0.9317 | Loss: 0.2135
Class-wise Accuracy:
  Class 0: 0.6276
  Class 1: 0.9894
  Class 2: 0.9545
  Class 3: 0.9743
  Class 4: 0.9684
  Class 5: 0.9776
  Class 6: 0.9875
  Class 7: 0.9553
  Class 8: 0.9138
  Class 9: 0.9604

[Round 4]
Test Accuracy: 0.9622 | Loss: 0.1379
Class-wise Accuracy:
  Class 0: 0.8602
  Class 1: 0.9938
  Class 2: 0.9738
  Class 3: 0.9901
  Class 4: 0.9827
  Class 5: 0.9753
  Class 6: 0.9864
  Cl

NameError: name 'target_accuracies' is not defined

In [13]:
[results[size]["def_target"][-1] for size in proxy_sizes]

[0.8704081632653061,
 0.9285714285714286,
 0.9581632653061225,
 0.9734693877551021,
 0.9806122448979592]

In [7]:
import torch
import torch.nn.functional as F
import copy
def distill_knowledge(global_model, local_models, proxy_loader, device, distill_epochs=3, temperature=3.0):
    print(f"→ Starting distillation with T = {temperature}")
    global_model.train()
    optimizer = torch.optim.SGD(global_model.parameters(), lr=0.01)

    for epoch in range(distill_epochs):
        print(f"  [Distill Epoch {epoch+1}/{distill_epochs}]")
        for images, _ in proxy_loader:
            images = images.to(device)
            ensemble_logits = torch.zeros((images.size(0), 10), device=device)

            with torch.no_grad():
                for model in local_models:
                    model.eval()
                    logits = model(images)
                    ensemble_logits += F.softmax(logits / temperature, dim=1)

            ensemble_logits /= len(local_models)
            output = global_model(images)
            student_log_probs = F.log_softmax(output / temperature, dim=1)
            loss = F.kl_div(student_log_probs, ensemble_logits, reduction="batchmean") * (temperature ** 2)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    return global_model


# Main FL loop with defense
global_model = MNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 10
num_clients = 5

for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=5,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=5,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    # Aggregation (FedAvg)
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    
    if rnd >= 3:
        proxy_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
        local_models = []
        for state in local_weights:
            local_model = MNISTCNN().to(device)
            local_model.load_state_dict(state)
            local_models.append(local_model)

        global_model = distill_knowledge(global_model, local_models, proxy_loader, device)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    
    d["def_overall"].append(acc)
    d["def_target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")



[Round 1]
Test Accuracy: 0.8151 | Loss: 0.5569
Class-wise Accuracy:
  Class 0: 0.0031
  Class 1: 0.9868
  Class 2: 0.9196
  Class 3: 0.9307
  Class 4: 0.9379
  Class 5: 0.9148
  Class 6: 0.9697
  Class 7: 0.8979
  Class 8: 0.7043
  Class 9: 0.8563

[Round 2]
Test Accuracy: 0.9044 | Loss: 0.2855
Class-wise Accuracy:
  Class 0: 0.4816
  Class 1: 0.9921
  Class 2: 0.9370
  Class 3: 0.9634
  Class 4: 0.9603
  Class 5: 0.9731
  Class 6: 0.9718
  Class 7: 0.9290
  Class 8: 0.8717
  Class 9: 0.9514

[Round 3]
Test Accuracy: 0.9264 | Loss: 0.2202
Class-wise Accuracy:
  Class 0: 0.6235
  Class 1: 0.9930
  Class 2: 0.9506
  Class 3: 0.9822
  Class 4: 0.9389
  Class 5: 0.9709
  Class 6: 0.9833
  Class 7: 0.9514
  Class 8: 0.8953
  Class 9: 0.9643

[Round 4]
→ Starting distillation with T = 3.0
  [Distill Epoch 1/3]
  [Distill Epoch 2/3]
  [Distill Epoch 3/3]
Test Accuracy: 0.9727 | Loss: 0.0907
Class-wise Accuracy:
  Class 0: 0.9827
  Class 1: 0.9877
  Class 2: 0.9806
  Class 3: 0.9743
  Class 4